In [16]:
# Cell [9] - Đọc thông tin tài khoản
with open('email_credentials.txt', 'r', encoding='utf-8') as file:
    account_info = file.readlines()
    sender_email = account_info[0].strip()  # Đọc email từ dòng đầu
    app_password = account_info[1].strip()  # Đọc App Password từ dòng thứ hai

# Cell [10] - Đọc nội dung email, người nhận, và bộ lọc
with open('email_content.txt', 'r', encoding='utf-8') as file:
    email_content = file.read()

with open('recipient.txt', 'r', encoding='utf-8') as file:
    recipient_email = file.read().strip()

with open('email_filter.txt', 'r', encoding='utf-8') as file:
    filter_sender = file.read().strip()

# Cell [11] - Gửi và nhận email
import smtplib
import imaplib
from email.mime.text import MIMEText

# Gửi email
try:
    msg = MIMEText(email_content, 'plain', 'utf-8')
    msg['Subject'] = 'Email Tự Động'
    msg['From'] = sender_email
    msg['To'] = recipient_email

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.send_message(msg)
    print("Gửi email thành công!")
except smtplib.SMTPAuthenticationError:
    print("Lỗi: Đăng nhập thất bại. Kiểm tra email và App Password.")
except smtplib.SMTPException as e:
    print(f"Lỗi SMTP: {e}")
except Exception as e:
    print(f"Lỗi khi gửi email: {e}")

# Nhận email
try:
    with imaplib.IMAP4_SSL('imap.gmail.com') as server:
        server.login(sender_email, app_password)
        server.select('INBOX')

        _, data = server.search(None, f'FROM "{filter_sender}"')
        if not data[0]:
            print(f"Không tìm thấy email từ {filter_sender}")
        else:
            for num in data[0].split()[:1]:
                _, msg_data = server.fetch(num, '(RFC822)')
                # Đảm bảo msg_data[0][1] là bytes
                if isinstance(msg_data[0][1], bytes):
                    email_msg = email.message_from_bytes(msg_data[0][1])
                    subject = email_msg['subject']
                    print(f"Tiêu đề email nhận được: {subject}")
                    if email_msg.is_multipart():
                        for part in email_msg.walk():
                            if part.get_content_type() == 'text/plain':
                                print(f"Nội dung: {part.get_payload(decode=True).decode('utf-8', errors='ignore')}")
                    else:
                        print(f"Nội dung: {email_msg.get_payload(decode=True).decode('utf-8', errors='ignore')}")
                else:
                    print(f"Lỗi: Dữ liệu không phải bytes: {type(msg_data[0][1])}")
except imaplib.IMAP4.error as e:
    print(f"Lỗi IMAP: {e}")
except Exception as e:
    print(f"Lỗi khi nhận email: {e}")

Gửi email thành công!
Tiêu đề email nhận được: =?UTF-8?B?UmU6IEVtYWlsIFThu7EgxJDhu5luZw==?=
Nội dung: Hello bạn tôi đã nhận đc email từ bạn nhukhiem

Vào Th 4, 7 thg 5, 2025 vào lúc 15:06 <nhukhiem24@gmail.com> đã viết:

> Chào bạn,
> Đây là email của tôi được gửi tự động bằng Python.
> Trân trọng.
>

